# Algorithmic Trading Pipeline
# Complete Workflow: Data → Model → Strategy → Backtest → Analysis

This notebook demonstrates the full pipeline for:
1. Loading/creating data catalog
2. Model hyperparameter optimization
3. Strategy hyperparameter optimization
4. Final backtest execution
5. Performance analysis and visualization

---
## Stage 0: Setup & Configuration
Load dependencies and configuration

In [2]:
# Use to debugging
%load_ext autoreload
%autoreload 2

# Core imports
from pathlib import Path
import yaml
import logging
import pandas as pd
import mlflow
from mlflow import MlflowClient
import tqdm as notebook_tqdm


# Nautilus Trader
from nautilus_trader.model.objects import Currency
from nautilus_trader.core.nautilus_pyo3 import CurrencyType
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.model.data import TradeTick

# Project modules
from engine.databento_loader import DatabentoTickLoader
from engine.hparam_tuner import OptunaHparamsTuner
from engine.performance_plots import (
    get_frequency_params, align_series,
    plot_balance_breakdown, plot_cumulative_returns,
    plot_rolling_sharpe, plot_underwater,
    plot_active_returns, plot_portfolio_allocation
)

# Setup logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
client = MlflowClient(tracking_uri="file:logs/mlflow")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Auto-launch MLflow UI if not running
"""
import socket
import subprocess

def is_port_in_use(port=5000):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

if not is_port_in_use(5000):
    print("🚀 Launching MLflow UI at http://localhost:5000")
    subprocess.Popen(
        ["mlflow", "ui", "--backend-store-uri", "file:logs/mlflow", "--port", "5000"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
else:
    print("✅ MLflow UI already running at http://localhost:5000")
"""

'\nimport socket\nimport subprocess\n\ndef is_port_in_use(port=5000):\n    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:\n        return s.connect_ex((\'localhost\', port)) == 0\n\nif not is_port_in_use(5000):\n    print("🚀 Launching MLflow UI at http://localhost:5000")\n    subprocess.Popen(\n        ["mlflow", "ui", "--backend-store-uri", "file:logs/mlflow", "--port", "5000"],\n        stdout=subprocess.DEVNULL,\n        stderr=subprocess.DEVNULL\n    )\nelse:\n    print("✅ MLflow UI already running at http://localhost:5000")\n'

In [ ]:
# Load Sapiens configuration
sapiens_cfg_path = Path("configs/sapiens_config.yaml")
sapiens_cfg = yaml.safe_load(sapiens_cfg_path.read_text(encoding="utf-8"))

# Load strategy and model configs
strategy_name = sapiens_cfg["SAPIENS_STRATEGY"]['strategy_name']
strategy_cfg_path = Path(f"strategies/{strategy_name}/strategy_config.yaml")
strategy_cfg = yaml.safe_load(strategy_cfg_path.read_text(encoding="utf-8"))["STRATEGY"]

model_name = sapiens_cfg["SAPIENS_MODEL"]['model_name']
model_cfg_path = Path(f"models/{model_name}/model_config.yaml")
model_cfg = yaml.safe_load(model_cfg_path.read_text(encoding="utf-8"))["MODEL"]


# Setup directories
logs_dir = Path(sapiens_cfg["logs_dir"])
logs_dir.mkdir(parents=True, exist_ok=True)

print("Configuration loaded successfully")
print(f"Model: {model_cfg['PARAMS']['model_name']}")
print(f"Strategy: {strategy_cfg['PARAMS']['strategy_name']}")
print(f"Backtest period: {sapiens_cfg['backtest_start']} to {sapiens_cfg['backtest_end']}")

Configuration loaded successfully
Model: UMIModel
Strategy: TopKStrategy
Backtest period: 2024-06-01 to 2024-12-01


---
## Stage 1: Data Catalog and HyperParameter Tuner setup
Load or create Nautilus Trader data catalog from Databento tick data

In [ ]:
# Configuration
FORCE_RELOAD_CATALOG = False  # Set to True to rebuild catalog
CATALOG_PATH = None  # Set custom path or None for default

# Initialize loader
logger.info("Initializing Databento loader...")
loader = DatabentoTickLoader(
    cfg=strategy_cfg["PARAMS"],
    venue_name=strategy_cfg["PARAMS"]["venue_name"]
)

# Determine catalog path
catalog_path = Path(CATALOG_PATH) if CATALOG_PATH else loader.catalog_path

# Load or create catalog
if not FORCE_RELOAD_CATALOG and loader.catalog_exists(catalog_path):
    logger.info(f"📂 Reusing existing catalog at: {catalog_path}")
    catalog = ParquetDataCatalog(path=str(catalog_path))
else:
    logger.info(f"🔄 Loading Databento ticks to catalog at: {catalog_path}")
    if FORCE_RELOAD_CATALOG:
        logger.info("Force reload enabled - rebuilding catalog")
    
    # Load with progress bar and memory management
    catalog = loader.load_to_catalog(
        catalog_path=catalog_path,
    )

# Add catalog path to config
strategy_cfg["PARAMS"]["catalog_path"] = str(catalog_path)

# Verify catalog
#instruments = catalog.instruments(instrument_type=TradeTick)  # takes too long on laptop. Use loader class instruments property instead
instruments = set(inst.id.value for inst in catalog.instruments())
print(f"\n✅ Catalog ready: {catalog.list_data_types()} data loaded")
print(f"Universe: {[str(symbol) for symbol in instruments]}")


2025-11-17 06:13:48,896 - __main__ - INFO - Initializing Databento loader...
2025-11-17 06:13:48,908 - __main__ - INFO - 📂 Reusing existing catalog at: C:\Users\gianc\Desktop\PYTHON\sapiens\data\nautilus_catalog



✅ Catalog ready: ['equity', 'trade_tick'] data loaded
Universe: ['SPY.XNAS', 'MSFT.XNAS', 'AAPL.XNAS', 'SGOV.XNAS', 'GOOGL.XNAS']


In [ ]:
# Initialize hyperparameter tuner
tuner = OptunaHparamsTuner(
    sapiens_config=sapiens_cfg,
    catalog=catalog,
    model_config=model_cfg,
    strategy_config=strategy_cfg,
    run_dir=logs_dir
)

print("Hyperparameter tuner initialized")
print(f"Model trials: {sapiens_cfg['SAPIENS_MODEL']['optimization']['n_trials']}")
print(f"Strategy trials: {sapiens_cfg['SAPIENS_STRATEGY']['optimization']['n_trials']}")

Hyperparameter tuner initialized
Model trials: 20
Strategy trials: 20


---
## Stage 2: Model Hyperparameter Optimization
Optimize model hyperparameters using Optuna

In [ ]:
# Run model hyperparameter optimization
logger.info("\n" + "="*70)
logger.info("🔬 STAGE 2: MODEL HYPERPARAMETER OPTIMIZATION")
logger.info("="*70 + "\n")

model_results = tuner.optimize_model()

print("\n✅ Model optimization complete!")
print(f"Best model path: {model_results['model_path']}")
print(f"MLflow run ID: {model_results['mlflow_run_id']}")
model_results['param_importance_fig'].show()

2025-11-17 06:13:49,265 - __main__ - INFO - 
2025-11-17 06:13:49,265 - __main__ - INFO - 🔬 STAGE 2: MODEL HYPERPARAMETER OPTIMIZATION
2025-11-17 06:13:49,265 - __main__ - INFO - ======================================================================

[I 2025-11-17 06:13:49,633] Using an existing study with name 'UMIModel' instead of creating a new one.
2025-11-17 06:13:49,655 - engine.hparam_tuner - INFO - Hyper-parameter tuning disabled. Taking best trial for UMIModel from database...
2025-11-17 06:13:50,157 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): config.mlflow-telemetry.io:443
2025-11-17 06:13:50,176 - engine.hparam_tuner - INFO - Best model trial: 10
2025-11-17 06:13:50,176 - engine.hparam_tuner - INFO - Best trial hparams: {'train_offset': '20B', 'inference_window': '18h', 'warm_training_epochs': 1, 'retrain_offset': '20D', 'n_retrains_on_valid': 0, 'lambda_ic': 0.25, 'lambda_sync': 0, 'lambda_rankic': 0, 'temperature': 0.09041726176785982, 'sync_thr': 


✅ Model optimization complete!
Best model path: logs\Models\UMIModel\trial_10\init.pt
MLflow run ID: 25db0812c6fa4463afecc698eeb6bc5d


2025-11-17 06:13:50,568 - urllib3.connectionpool - DEBUG - https://config.mlflow-telemetry.io:443 "GET /3.5.1.json HTTP/1.1" 200 215


2025-11-17 06:13:51,670 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.mlflow-telemetry.io:443
2025-11-17 06:13:52,572 - urllib3.connectionpool - DEBUG - https://api.mlflow-telemetry.io:443 "POST /log HTTP/1.1" 200 30


---
## Stage 3: Strategy Hyperparameter Optimization
Optimize strategy hyperparameters using best model from Stage 2

In [ ]:
# Use to debugging
%load_ext autoreload
%autoreload 2


# Run strategy hyperparameter optimization
logger.info("\n" + "="*70)
logger.info("📊 STAGE 3: STRATEGY HYPERPARAMETER OPTIMIZATION")
logger.info("="*70 + "\n")

model_name = model_cfg['PARAMS']['model_name']
strategy_results = tuner.optimize_strategy(model_name=model_name)
strategy_results['param_importance_fig'].show()

print("\n✅ Strategy optimization complete!")
print(f"Best hyperparameters: {strategy_results['hparams']}")
print(f"\nBest metrics:")
for metric, value in strategy_results['metrics'].items():
    print(f"  {metric}: {value:.4f}")
print(f"\nMLflow run ID: {strategy_results['mlflow_run_id']}")

2025-11-17 06:14:17,654 - __main__ - INFO - 
2025-11-17 06:14:17,655 - __main__ - INFO - 📊 STAGE 3: STRATEGY HYPERPARAMETER OPTIMIZATION
2025-11-17 06:14:17,656 - __main__ - INFO - ======================================================================



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[I 2025-11-17 06:14:18,183] Using an existing study with name 'TopKStrategy_UMIModel' instead of creating a new one.
2025-11-17 06:14:18,193 - engine.hparam_tuner - INFO - Hyper-parameter tuning for strategy disabled. Taking best trial for TopKStrategy from database...
2025-11-17 06:14:18,205 - engine.hparam_tuner - INFO - Considering best trial for total_pnl_pct as the overall multi-objective best trial
c:\Users\gianc\Desktop\PYTHON\sapiens\.venv\Lib\site-packages\optuna\visualization\_utils.py:67: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.

2025-11-17 06:14:18,284 - engine.hparam_tuner - INFO - Parameter importances: {'top_k': np.float64(1.0), 'optimizer_name': np.float64(0.0)}
2025-11-17 06:14:18,284 - engine.hparam_tuner - INFO - Best strategy trial: 18
2025-11-17 06:14:18,284 - engine.hparam_tuner - INFO - Best strategy hparams: {'top_k': 3, 'optimizer_name': 'm2'}
2025-11-17 06:14:18,299 - engine.hparam_tuner - INFO - Best stra


✅ Strategy optimization complete!
Best hyperparameters: {'top_k': 3, 'optimizer_name': 'm2'}

Best metrics:
  returns_volatility: 0.1111
  avg_return: 0.0004
  avg_loss_pct: -0.0018
  avg_win_pct: 0.0022
  sharpe_ratio: 7.1031
  sortino_ratio: 18.6550
  profit_factor: 1.4979
  risk_return_ratio: 0.1285
  total_pnl: -297.5900
  total_pnl_pct: -1.4880
  max_drawdown: 0.0000
  max_winner: 57.9400
  avg_winner: 13.1932
  min_winner: 0.0100
  min_loser: -0.1100
  avg_loser: -14.0399
  max_loser: -78.5200
  expectancy: -2.3999
  win_rate: 0.4274
  num_trades: 4.0000

MLflow run ID: 403e29685776437885ed943d025984b7


2025-11-17 06:14:22,738 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.mlflow-telemetry.io:443
2025-11-17 06:14:23,619 - urllib3.connectionpool - DEBUG - https://api.mlflow-telemetry.io:443 "POST /log HTTP/1.1" 200 30


---
## Stage 4: Final Backtest
Run final backtest on full period with optimized hyperparameters

In [ ]:
# Get optimization context
run = client.get_run(strategy_results["mlflow_run_id"])
optimization_id = run.data.tags.get("optimization_id", "")

# Define backtest period
backtest_start = sapiens_cfg["backtest_start"]
backtest_end = sapiens_cfg["backtest_end"]

print(f"Running final backtest: {backtest_start} to {backtest_end}")

In [ ]:
# Execute final backtest
logger.info("\n" + "="*70)
logger.info("🚀 STAGE 4: FINAL BACKTEST")
logger.info("="*70 + "\n")

final_metrics, final_time_series = tuner.run_final_backtest(
    backtest_start=backtest_start,
    backtest_end=backtest_end,
    strategy_hpo_run_id=strategy_results["mlflow_run_id"],
    optimization_id=optimization_id
)

print("\n✅ Final backtest complete!")
print("\nFinal Performance Metrics:")
print("="*50)
for metric, value in sorted(final_metrics.items()):
    print(f"{metric:.<40} {value:>10.4f}")

---
## Stage 5: Performance Analysis
Detailed analysis and visualization of backtest results

In [ ]:
# Load results from MLflow

# Get the most recent backtest run
exp = client.get_experiment_by_name("Backtests")
if not exp:
    exp_id = client.create_experiment("Backtests")
else:
    exp_id = exp.experiment_id
runs = client.search_runs(
    experiment_ids=[exp_id],
    order_by=["start_time DESC"],
    max_results=1
)
backtest_run = runs[0]
backtest_run_id = backtest_run.info.run_id

print(f"Loading backtest run: {backtest_run_id}")

In [ ]:
# Load account and positions data
acc_path = client.download_artifacts(run_id=backtest_run_id, path="account_report.json")
pos_path = client.download_artifacts(run_id=backtest_run_id, path="positions_report.json")

account_df = pd.read_csv(acc_path, index_col=0, parse_dates=True)
positions_df = pd.read_csv(pos_path, index_col=0, parse_dates=True)

print(f"Account snapshots: {len(account_df)}")
print(f"Position records: {len(positions_df)}")

In [ ]:
# Calculate returns
freq = strategy_cfg['PARAMS']['freq']
freq_params = get_frequency_params(freq)

# Extract portfolio values
currency_code = strategy_cfg['PARAMS']['currency']
portfolio_values = account_df[account_df['currency'] == currency_code]['total']
portfolio_values = portfolio_values.resample(freq_params['resample_freq']).last().ffill()
strategy_ret = portfolio_values.pct_change().fillna(0)

# Load benchmark and risk-free data
data_dict = tuner.get_ohlcv_data_from_catalog(
    frequency=freq,
    start=pd.Timestamp(backtest_start),
    end=pd.Timestamp(backtest_end),
    instrument_ids=[strategy_cfg['PARAMS']['benchmark_ticker'],
                    strategy_cfg['PARAMS']['risk_free_ticker']]
)
benchmark_ret = data_dict[strategy_cfg['PARAMS']['benchmark_ticker']]['close'].pct_change()
rf_ret = data_dict[strategy_cfg['PARAMS']['risk_free_ticker']]['close'].pct_change()

# Align series
strategy_ret, benchmark_ret, rf_ret = align_series(
    strategy_ret, benchmark_ret, rf_ret, freq_params['resample_freq']
)

print(f"Returns calculated for {len(strategy_ret)} periods")

### 5.1: Balance Over Time

In [ ]:
fig = plot_balance_breakdown(
    account_df=account_df,
    resample_freq=freq_params['resample_freq']
)

### 5.2: Cumulative Returns

In [ ]:
fig = plot_cumulative_returns(
    strategy_ret=strategy_ret,
    benchmark_ret=benchmark_ret
)

### 5.3: Rolling Sharpe Ratio

In [ ]:
window = max(10, int(freq_params['periods_per_year'] / 12))
fig = plot_rolling_sharpe(
    strategy_ret=strategy_ret,
    benchmark_ret=benchmark_ret,
    rf_ret=rf_ret,
    window=window,
    annualization_factor=freq_params['annualization_factor']
)

### 5.4: Drawdown Analysis

In [ ]:
fig = plot_underwater(strategy_ret=strategy_ret)

### 5.5: Active Returns

In [ ]:
fig = plot_active_returns(
    strategy_ret=strategy_ret,
    benchmark_ret=benchmark_ret,
    freq=freq
)

### 5.6: Portfolio Allocation

In [ ]:
"""
fig = plot_portfolio_allocation(
    positions_df=positions_df,
    resample_freq=freq_params['resample_freq']
)
"""

### 5.7: Summary Statistics

In [ ]:
# Calculate summary statistics
total_return = (1 + strategy_ret).prod() - 1
annualized_return = (1 + total_return) ** (freq_params['periods_per_year'] / len(strategy_ret)) - 1
annualized_vol = strategy_ret.std() * freq_params['annualization_factor']
sharpe_ratio = (strategy_ret.mean() - rf_ret.mean()) / strategy_ret.std() * freq_params['annualization_factor']

# Drawdown
cumulative = (1 + strategy_ret).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max
max_drawdown = drawdown.min()

# Win rate
winning_periods = (strategy_ret > 0).sum()
win_rate = winning_periods / len(strategy_ret)

# Create summary DataFrame
summary = pd.DataFrame({
    'Metric': [
        'Total Return (%)',
        'Annualized Return (%)',
        'Annualized Volatility (%)',
        'Sharpe Ratio',
        'Max Drawdown (%)',
        'Win Rate (%)',
        'Number of Periods'
    ],
    'Value': [
        f"{total_return * 100:.2f}",
        f"{annualized_return * 100:.2f}",
        f"{annualized_vol * 100:.2f}",
        f"{sharpe_ratio:.2f}",
        f"{max_drawdown * 100:.2f}",
        f"{win_rate * 100:.2f}",
        len(strategy_ret)
    ]
})

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(summary.to_string(index=False))

---
## Comparison Matrix: All Models × Strategies

In [ ]:
# Generate HPO results matrix
hpo_matrix = tuner.get_strategy_hpo_matrix(metric="total_pnl_pct")
print("\nStrategy HPO Results Matrix (total_pnl_pct):")
print(hpo_matrix)

# Generate final backtest results matrix
backtest_matrix = tuner.get_final_backtest_matrix(metric="sharpe_ratio")
print("\nFinal Backtest Results Matrix (sharpe_ratio):")
print(backtest_matrix)

---
## Pipeline Complete ✅

**Next Steps:**
- Review MLflow UI: `mlflow ui --backend-store-uri logs/mlflow`
- Explore experiment tracking and compare runs
- Adjust hyperparameters in `configs/config.yaml` and rerun
- Export results for production deployment